In [6]:
%store -r

In [2]:
import os
import pathlib
from glob import glob
import requests

from math import floor, ceil

import earthaccess
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr
import xrspatial

c:\Users\tjsto\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Climate Data
At least one climate variable from the MACAv2 THREDDS data server. Your project should compare two climate scenarios of your choice (e.g. different time periods, different emission scenarios). You can find a tutorial on how to access these climate data on earthdatascience.org

In [3]:
def convert_longitude(longitude):
    """Convert longitude range from 0-360 to [-180:180]"""
    return (longitude - 360) if longitude > 180 else longitude

In [ ]:
start_year = 2081
if start_year == 2096:
    end_year = 2099
else:
    end_year = start_year + 4
# end_year = 2099 if start_year == 2096 else (start_year + 4)
print(end_year)
maca_url = (
        'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM'
        f'/macav2metdata_{variable}_BNU-ESM_r1i1p1_'
        f'{scenario}_{start_year}_{end_year}_CONUS_monthly.nc')

print(maca_url)

2085
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2081_2085_CONUS_monthly.nc


In [4]:
def MACAv2_data(variable, scenario, start_year, site_gdf):
    end_year = 2099 if start_year == 2096 else (start_year + 4)
    maca_url = (
        'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/BNU-ESM'
        f'/macav2metdata_{variable}_BNU-ESM_r1i1p1_'
        f'{scenario}_{start_year}_{end_year}_CONUS_monthly.nc')
    maca_da = xr.open_dataset(maca_url).squeeze().precipitation
    bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
    # Reassign coordinates to [-180:180]
    maca_da = maca_da.assign_coords(
        lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
    )
    maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    maca_da = maca_da.rio.clip_box(*bounds)


    return maca_da


In [ ]:
# Test case for function
MACAv2_data('pr', 'rcp85', 2096, JNF_gdf)

In [10]:
variable_list = ['pr']
scenario_list = ['rcp45', 'rcp85']
start_year_list = [2096]

maca_da_list = []
for site in site_list:
    for variable in variable_list:
        for scenario in scenario_list:
            for start_year in start_year_list:
                maca_da = MACAv2_data(variable, scenario, start_year, site)

                maca_da_list.append(dict(
                    site_name=site.FORESTNAME.values[0],
                    variable=variable,
                    scenario=scenario,
                    start_year=start_year,
                    da=maca_da))

maca_da_list

[{'site_name': 'George Washington National Forest',
  'variable': 'pr',
  'scenario': 'rcp45',
  'start_year': 2096,
  'da': <xarray.DataArray 'precipitation' (time: 48, lat: 40, lon: 50)> Size: 384kB
  [96000 values with dtype=float32]
  Coordinates:
    * lat      (lat) float64 320B 37.52 37.56 37.6 37.65 ... 39.06 39.1 39.15
    * time     (time) object 384B 2096-01-15 00:00:00 ... 2099-12-15 00:00:00
    * lon      (lon) float64 400B -80.31 -80.27 -80.23 ... -78.36 -78.31 -78.27
      crs      int64 8B 0
  Attributes:
      long_name:      Monthly Precipitation Amount
      units:          mm
      standard_name:  precipitation
      cell_methods:   time: sum(interval: 24 hours): sum over days
      comments:       Total monthly precipitation at surface: includes both liq...
      _ChunkSizes:    [  8 115 276]},
 {'site_name': 'George Washington National Forest',
  'variable': 'pr',
  'scenario': 'rcp85',
  'start_year': 2096,
  'da': <xarray.DataArray 'precipitation' (time: 48, la

In [11]:
MACAv2_df = pd.DataFrame(maca_da_list)
MACAv2_df

,site_name,variable,scenario,start_year,da
0,George Washington National Forest,pr,rcp45,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
1,George Washington National Forest,pr,rcp85,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
2,Jefferson National Forest,pr,rcp45,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
3,Jefferson National Forest,pr,rcp85,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
